# Value at Risk

Beta and standard deviation are ways to measure risk however neither of them consider the probability distribution.

VAR measures the amount of potential loss that could happen in an investment over a given period in time.

## Variance Method

Assumes return are normally distributed

$$ VaR = position*(\mu_{period} - z\sigma_{period}) $$
$$ \mu_{period} = \mu_{daily}n $$
$$\sigma_{period} = \sigma_{daily} \sqrt{n} $$

In [11]:
import pandas as pd
import numpy as np
import requests
from scipy.stats import norm

In [5]:
def value_at_risk(S, c, mu, sigma, n):
    z_stat = norm.ppf(1-c)
    return S*(mu*n - z_stat*sigma*n**0.5)

In [6]:
def get_data(stock, start, end):
    
    key = '*'
    
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={stock}&apikey={key}&outputsize=full"
    
    r = requests.get(url)
    r.raise_for_status()
    
    df = pd.DataFrame.from_dict(
        r.json()['Time Series (Daily)'], orient='index'
    )
    
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    df['5. adjusted close'] = df['5. adjusted close'].astype('float')
    
    return df[start:end]['5. adjusted close']

In [7]:
data = get_data('C', '2014-01-01', '2017-10-15')

In [12]:
S = 1e6 #poistion
c=0.99 #confidence interval
mu = np.mean(data.pct_change())
sigma = np.std(data.pct_change())

if we have a million dollars in citi group shares then 99% of the time we should not lose more than \\$36,382 tommorow and \\$211,083 in 30 day time.

In [13]:
value_at_risk(S,c,mu,sigma,1)

36382.70696897758

In [14]:
value_at_risk(S,c,mu,sigma,30)

211083.75008959114

## Monte Carlo approach

In [15]:
iterations = 1000000

In [21]:
def value_at_risk_monte_carlo(S, c, mu, sigma, n, iterations):
    
    rand = np.random.normal(0,1,iterations)
    
    #simulate stock price using gbm
    stock_price = S*np.exp(n*(mu - 0.5*sigma**2)+sigma*np.sqrt(n)*rand)
    
    stock_price = np.array(stock_price)
    
    return S - np.quantile(stock_price, 1-c)
    

In [22]:
value_at_risk_monte_carlo(S, c, mu, sigma, 1, iterations)

34948.21398076054